In [1]:
import dotenv
import typing

dotenv.load_dotenv(dotenv.find_dotenv(usecwd=True))

True

In [2]:
import getpass
import os

def _set_if_undefined(var: str):
    if os.environ.get(var) is None:
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("OPENAI_API_KEY")

In [3]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

/Users/jake.wood/Documents/CKO/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from agentc.catalog import Catalog

catalog = Catalog()
application_span = catalog.Span(
    name="PatientIQ Medical Researcher",
)

In [5]:
from agentc_langgraph.agent import ReActAgent
from langchain_core.messages import SystemMessage
from langgraph.graph import END


class ResearchAgent(ReActAgent):
    def __init__(self, span):
        super().__init__(catalog=catalog, prompt_name="medical_researcher", span=span, chat_model=llm)

    def _invoke(self, span, state, config):
        agent = self.create_react_agent(span)
        result = agent.invoke(state)
        state["messages"] += [SystemMessage(content=result["messages"][-1].content, name="researcher")]
        return result

In [6]:
from agentc_langgraph.agent import State
from agentc_langgraph.graph import GraphRunnable
from langgraph.graph import START
from langgraph.graph import StateGraph


def researcher_out_edges(state) -> typing.Literal["__end__"]:
    if "FINAL ANSWER" in state["messages"][-1].content:
        return END
    else:
        return "__end__"


class MyResearcherApplication(GraphRunnable):
    def __init__(self):
        super().__init__(catalog=catalog, span=application_span)

    def compile(self):
        workflow = StateGraph(State)
        workflow.add_node("researcher", ResearchAgent(span=self.span))
        workflow.add_edge(START, "researcher")
        workflow.add_conditional_edges("researcher", researcher_out_edges)
        return workflow.compile()


graph = MyResearcherApplication()

In [8]:
from IPython.display import Image
from IPython.display import display

display(Image(graph.get_graph().draw_mermaid_png()))

LookupError: Catalog version not found for kind = 'prompt'! Please run 'agentc publish' to create the catalog.

In [ ]:
from agentc.span import UserContent

user_input = "Hello, can you tell me anything about asthma?"
application_span.log(content=UserContent(value=user_input))
events = graph.stream(
    {"messages": [("user", user_input)], "is_last_step": False, "previous_node": None},
    # Maximum number of steps to take in the graph
    {"recursion_limit": 10},
)

for event in events:
    for key in event:
        for msg in event[key]["messages"]:
            if isinstance(msg, SystemMessage):
                print(msg.content)
                print("----")